<font color=red>This is a draft version and the notebook is incomplete and the statements do not match the data at this point. It will be fixed soon.</font>

# SVMs and Kernels

In this notebook you will fit a Support Vector Machine (SVM) classifer to data using scikit-learn. You will use different kernels and see how kernels produce nonlinear decision surfaces. You will also predict the labels for datapoints and measure the performance of the SVM.

## Before you start

- In order for the notebooks to function as intended, modify only between lines marked "### begin your code here (__ lines)." and "### end your code here.". 

- The line count is a suggestion of how many lines of code you need to accomplish what is asked.

- You should execute the cells (the boxes that a notebook is composed of) in order.

- You can execute a cell by pressing Shift and Enter (or Return) simultaneously.

- You should have completed the previous Jupyter notebooks before attempting this one as the concepts covered there are not repeated, for the sake of brevity.

## Loading the appropriate packages

We will import SVM classifier class (SVC) as well some other packages you will use.

In [1]:
import numpy as np
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
import pandas as pd
import plotly.graph_objs as go

Let's turn off the scientific notation for floating point numbers.

In [2]:
np.set_printoptions(suppress=True)

## Loading and examining the data

We will load our data from a CSV file and put it in a pandas an object of the `DataFrame` class.

This dataset is the breast cancer Wisconsin (diagnostic) dataset which contains 30 different features computed from a images of a fine needle aspirate (FNA) of breast masses for 569 patients with each example labeled as being a _benign_ or _malignant_ mass.

* This was taken and modified from the Machine Learning dataset repository of School of Information and Computer Science of University of California Irvine (UCI):
 
> _Dua, D. and Graff, C. (2019). UCI Machine Learning Repository [http://archive.ics.uci.edu/ml]. Irvine, CA: University of California, School of Information and Computer Science._

In [3]:
df = pd.read_csv('/Users/ashis/Desktop/new breast.csv')

Let's take a look at the data:

In [4]:
df

,id,type,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,...,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst
0,8510426,B,13.540,14.36,87.46,566.3,0.09779,0.08129,0.06664,0.04781,...,15.11,19.26,99.70,711.2,0.1440,0.17730,0.23900,0.12880,0.2977,0.07259
1,8510653,B,13.080,15.71,85.63,520.0,0.10750,0.12700,0.04568,0.03110,...,14.50,20.49,96.09,630.5,0.1312,0.27760,0.18900,0.07283,0.3184,0.08183
2,8510824,B,9.504,12.44,60.34,273.9,0.10240,0.06492,0.02956,0.02076,...,10.23,15.66,65.13,314.9,0.1324,0.11480,0.08867,0.06227,0.2450,0.07773
3,8511133,M,15.340,14.26,102.50,704.4,0.10730,0.21350,0.20770,0.09756,...,18.07,19.08,125.10,980.9,0.1390,0.59540,0.63050,0.23930,0.4667,0.09946
4,851509,M,21.160,23.04,137.20,1404.0,0.09428,0.10220,0.10970,0.08632,...,29.17,35.59,188.00,2615.0,0.1401,0.26000,0.31550,0.20090,0.2822,0.07526
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
77,862717,M,13.610,24.98,88.05,582.7,0.09488,0.08511,0.08625,0.04489,...,16.99,35.27,108.60,906.5,0.1265,0.19430,0.31690,0.11840,0.2651,0.07397
78,862722,B,6.981,13.43,43.79,143.5,0.11700,0.07568,0.00000,0.00000,...,7.93,19.54,50.41,185.2,0.1584,0.12020,0.00000,0.00000,0.2932,0.09382
79,862965,B,12.180,20.52,77.22,458.7,0.08013,0.04038,0.02383,0.01770,...,13.34,32.84,84.58,547.8,0.1123,0.08862,0.11450,0.07431,0.2694,0.06878
80,862980,B,9.876,19.40,63.95,298.3,0.10050,0.09697,0.06154,0.03029,...,10.76,26.83,72.22,361.2,0.1559,0.23020,0.26440,0.09749,0.2622,0.08490


To do that we first need to extract our data from the dataframe in NumPy arrays. We use `LabelEncoder` from scikit-learn to transform labels into $\{-1,+1\}$:

In [5]:
X = df.drop('type', axis=1).to_numpy()
y_text = df['type'].to_numpy()
y = (2 * LabelEncoder().fit_transform(y_text)) - 1

Let's check `X`, `y_text` and `y`:

In [6]:
X

array([[8510426.     ,      13.54   ,      14.36   , ...,       0.1288 ,
              0.2977 ,       0.07259],
       [8510653.     ,      13.08   ,      15.71   , ...,       0.07283,
              0.3184 ,       0.08183],
       [8510824.     ,       9.504  ,      12.44   , ...,       0.06227,
              0.245  ,       0.07773],
       ...,
       [ 862965.     ,      12.18   ,      20.52   , ...,       0.07431,
              0.2694 ,       0.06878],
       [ 862980.     ,       9.876  ,      19.4    , ...,       0.09749,
              0.2622 ,       0.0849 ],
       [ 862989.     ,      10.49   ,      19.29   , ...,       0.03203,
              0.2826 ,       0.07552]])

In [7]:
X.shape

(82, 31)

Let's do the same thing for `y_text`:

In [8]:
y_text

array(['B', 'B', 'B', 'M', 'M', 'M', 'M', 'M', 'M', 'M', 'M', 'M', 'M',
       'M', 'M', 'M', 'M', 'M', 'B', 'M', 'M', 'M', 'M', 'B', 'M', 'B',
       'B', 'B', 'B', 'B', 'M', 'M', 'B', 'M', 'M', 'B', 'B', 'B', 'B',
       'M', 'B', 'M', 'M', 'B', 'B', 'B', 'B', 'M', 'B', 'M', 'M', 'B',
       'M', 'B', 'M', 'M', 'B', 'B', 'B', 'M', 'M', 'B', 'M', 'M', 'M',
       'B', 'B', 'B', 'M', 'B', 'B', 'M', 'M', 'B', 'B', 'B', 'M', 'M',
       'B', 'B', 'B', 'B'], dtype=object)

...and for shape of `y_text`:

In [9]:
y_text.shape

(82,)

Finally, let's check `y`:

In [10]:
y

array([-1, -1, -1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
        1, -1,  1,  1,  1,  1, -1,  1, -1, -1, -1, -1, -1,  1,  1, -1,  1,
        1, -1, -1, -1, -1,  1, -1,  1,  1, -1, -1, -1, -1,  1, -1,  1,  1,
       -1,  1, -1,  1,  1, -1, -1, -1,  1,  1, -1,  1,  1,  1, -1, -1, -1,
        1, -1, -1,  1,  1, -1, -1, -1,  1,  1, -1, -1, -1, -1])

In [11]:
y.shape

(82,)

Let's also do a scatter plot of our data:

In [12]:
points_colorscale = [
                     [0.0, 'rgb(239, 85, 59)'],
                     [1.0, 'rgb(99, 110, 250)'],
                    ]

points = go.Scatter(
                    x=df['Feature 1'],
                    y=df['Feature 2'],
                    mode='markers',
                    marker=dict(color=y,
                                colorscale=points_colorscale)
                   )
layout = go.Layout(
                   xaxis=dict(range=[-1.05, 1.05]),
                   yaxis=dict(range=[-1.05, 1.05])
                  )

fig = go.Figure(data=[points], layout=layout)
fig.show()

KeyError: 'Feature 1'

## Splitting data

Let's split our data into training, validation and test sets. Let's use 60% for training, 20% for validation and  20% for test data.

In [12]:
(X_train, X_vt, y_train, y_vt) = train_test_split(X, y, test_size=0.4, random_state=0)
(X_validation, X_test, y_validation, y_test) = train_test_split(X_vt, y_vt, test_size=0.5, random_state=0)

## Building and visualizing a SVM

Finally, let's build our SVM. We will use the `SVC` class from scikit-learn. For now, we are not using kernels, so we should set the `kernel` argument of `SVC` to `'linear'`. We don't need to specify any other parameters for now. You can find the documentation for `SVC` here:

https://scikit-learn.org/stable/modules/generated/sklearn.svm.SVC.html

So, make an object of class `SVC` and assign it to the name `svm`:

In [13]:
### begin your code here (1 line).
svm = SVC(kernel = 'linear')
### end your code here.

Now, fit `svm` to `X_train` and `y_train`:

In [14]:
### begin your code here (1 line).
svm.fit(X_train,y_train)
### end your code here.

SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='linear',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

You will get a summary for the model:

> SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
>   decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
>   kernel='rbf', max_iter=-1, probability=False, random_state=None,
>   shrinking=True, tol=0.001, verbose=False)

* You may also get a warning because you have not explicitly set gamma and the default setting for that is going to change in newer versions of scikit-learn. Don't worry about that warniong.

Let's visualize the decision surface our `svm` with its supprt vectors:

In [15]:
decision_colorscale = [
                       [0.0, 'rgb(239,  85,  59)'],
                       [0.5, 'rgb(  0,   0,   0)'],
                       [1.0, 'rgb( 99, 110, 250)']
                      ]

detail_steps = 100

(x_vis_0_min, x_vis_1_min) = (-1.05, -1.05) #X_train.min(axis=0)
(x_vis_0_max, x_vis_1_max) = ( 1.05,  1.05) #X_train.max(axis=0)

x_vis_0_range = np.linspace(x_vis_0_min, x_vis_0_max, detail_steps)
x_vis_1_range = np.linspace(x_vis_1_min, x_vis_1_max, detail_steps)

(XX_vis_0, XX_vis_1) = np.meshgrid(x_vis_0_range, x_vis_0_range)

X_vis = np.c_[XX_vis_0.reshape(-1), XX_vis_1.reshape(-1)]

YY_vis = svm.decision_function(X_vis).reshape(XX_vis_0.shape)

points = go.Scatter(
                    x=df['Feature 1'],
                    y=df['Feature 2'],
                    mode='markers',
                    marker=dict(
                                color=y,
                                colorscale=points_colorscale),
                    showlegend=False
                   )
SVs = svm.support_vectors_
support_vectors = go.Scatter(
                             x=SVs[:, 0],
                             y=SVs[:, 1],
                             mode='markers',
                             marker=dict(
                                         size=15,
                                         color='black',
                                         opacity = 0.1,
                                         colorscale=points_colorscale),
                             line=dict(dash='solid'),
                             showlegend=False
                            )

decision_surface = go.Contour(x=x_vis_0_range,
                              y=x_vis_1_range,
                              z=YY_vis,
                              contours_coloring='lines',
                              line_width=2,
                              contours=dict(
                                            start=0,
                                            end=0,
                                            size=1),
                              colorscale=decision_colorscale,
                              showscale=False
                             )

margins = go.Contour(x=x_vis_0_range,
                     y=x_vis_1_range,
                     z=YY_vis,
                     contours_coloring='lines',
                     line_width=2,
                     contours=dict(
                                   start=-1,
                                   end=1,
                                   size=2),
                     line=dict(dash='dash'),
                     colorscale=decision_colorscale,
                     showscale=False
                    )

fig2 = go.Figure(data=[margins, decision_surface, support_vectors, points], layout=layout)
fig2.show()

ValueError: X.shape[1] = 2 should be equal to 31, the number of features at training time

The datapoints, the decision surface (which is a line here), the margins and the support vectors are shown in the plot.

## Kernels

As you can see, the decision surface is underfiiting the data. Let's use a polynomial kernel. Define `svm_p2` to be an instance of class `SVC` but this time with arguments `kernel='poly'` and `degree=2` to define a degree-2 polynomial kernel:

In [16]:
### begin your code here (1 line).
svm_p2 = SVC(kernel = 'poly', degree = 2)
### end your code here.

..and fit it to your training data:

In [18]:
### begin your code here (1 line).
svm_p2.fit(X_train, y_train)
### end your code here.

SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=2, gamma='scale', kernel='poly',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

You will get a summary of your model:
    
> SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
>   decision_function_shape='ovr', degree=2, gamma='auto_deprecated',
>   kernel='poly', max_iter=-1, probability=False, random_state=None,
>   shrinking=True, tol=0.001, verbose=False)

Now, let's visualize this model:

In [19]:
YY_vis_p2 = svm_p2.decision_function(X_vis).reshape(XX_vis_0.shape)

SVs_p2 = svm_p2.support_vectors_
support_vectors_p2 = go.Scatter(
                                x=SVs_p2[:, 0],
                                y=SVs_p2[:, 1],
                                mode='markers',
                                marker=dict(
                                            size=15,
                                            color='black',
                                            opacity = 0.1,
                                            colorscale=points_colorscale),
                                line=dict(dash='solid'),
                                showlegend=False
                               )

decision_surface_p2 = go.Contour(x=x_vis_0_range,
                                 y=x_vis_1_range,
                                 z=YY_vis_p2,
                                 contours_coloring='lines',
                                 line_width=2,
                                 contours=dict(
                                               start=0,
                                               end=0,
                                               size=1),
                                 colorscale=decision_colorscale,
                                 showscale=False
                                )

margins_p2 = go.Contour(x=x_vis_0_range,
                        y=x_vis_1_range,
                        z=YY_vis_p2,
                        contours_coloring='lines',
                        line_width=2,
                        contours=dict(
                                      start=-1,
                                      end=1,
                                      size=2),
                        line=dict(dash='dash'),
                        colorscale=decision_colorscale,
                        showscale=False
                       )

fig3 = go.Figure(data=[margins_p2, decision_surface_p2, support_vectors_p2, points], layout=layout)
fig3.show()

ValueError: X.shape[1] = 2 should be equal to 31, the number of features at training time

Looks much better. But let's try a degree 3 model. Define `svm_p3` like `svm_p2` but with `degree=3` this time:

In [20]:
### begin your code here (1 line).
svm_p3 = SVC(degree = 3)
### end your code here.

Next, fit your `svm_p3` model to the training data:

In [21]:
### begin your code here (1 line).
svm_p3.fit(X_train, y_train)
### end your code here.

SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

Your model summary will be:

> SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
>   decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
>   kernel='poly', max_iter=-1, probability=False, random_state=None,
>   shrinking=True, tol=0.001, verbose=False)

Let's visualize `svm_p3`:

In [22]:
YY_vis_p3 = svm_p3.decision_function(X_vis).reshape(XX_vis_0.shape)

SVs_p3 = svm_p3.support_vectors_
support_vectors_p3 = go.Scatter(
                                x=SVs_p3[:, 0],
                                y=SVs_p3[:, 1],
                                mode='markers',
                                marker=dict(
                                            size=15,
                                            color='black',
                                            opacity = 0.1,
                                            colorscale=points_colorscale),
                                line=dict(dash='solid'),
                                showlegend=False
                               )

decision_surface_p3 = go.Contour(x=x_vis_0_range,
                                 y=x_vis_1_range,
                                 z=YY_vis_p3,
                                 contours_coloring='lines',
                                 line_width=2,
                                 contours=dict(
                                               start=0,
                                               end=0,
                                               size=1),
                                 colorscale=decision_colorscale,
                                 showscale=False
                                )

margins_p3 = go.Contour(x=x_vis_0_range,
                        y=x_vis_1_range,
                        z=YY_vis_p3,
                        contours_coloring='lines',
                        line_width=2,
                        contours=dict(
                                      start=-1,
                                      end=1,
                                      size=2),
                        line=dict(dash='dash'),
                        colorscale=decision_colorscale,
                        showscale=False
                       )

fig4 = go.Figure(data=[margins_p3, decision_surface_p3, support_vectors_p3, points], layout=layout)
fig4.show()

ValueError: X.shape[1] = 2 should be equal to 31, the number of features at training time

Let's try a RBF (Radial Basis Function) kernel as well. RBFs are the default kernel for scikit-learn's SVC. So build a model `svm_r` with either `kernel=rbf` argument setting or just skip the `kernel` (also the `degree` argument is uselss here, since we are not using a polynomial kernel, so just skip that): 

In [24]:
### begin your code here (1 line).
svm_r = SVC(kernel = 'rbf')
### end your code here.

Fit your `svm_r` model to the training data as well:

In [25]:
### begin your code here (1 line).
svm_r.fit(X_train, y_train)
### end your code here.

SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

This will be the parameter summary:

> SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
>   decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
>   kernel='rbf', max_iter=-1, probability=False, random_state=None,
>   shrinking=True, tol=0.001, verbose=False)

We will visualize this model as well:

In [26]:
YY_vis_r = svm_r.decision_function(X_vis).reshape(XX_vis_0.shape)

SVs_r = svm_r.support_vectors_
support_vectors_r = go.Scatter(
                                x=SVs_r[:, 0],
                                y=SVs_r[:, 1],
                                mode='markers',
                                marker=dict(
                                            size=15,
                                            color='black',
                                            opacity = 0.1,
                                            colorscale=points_colorscale),
                                line=dict(dash='solid'),
                                showlegend=False
                               )

decision_surface_r = go.Contour(x=x_vis_0_range,
                                 y=x_vis_1_range,
                                 z=YY_vis_r,
                                 contours_coloring='lines',
                                 line_width=2,
                                 contours=dict(
                                               start=0,
                                               end=0,
                                               size=1),
                                 colorscale=decision_colorscale,
                                 showscale=False
                                )

margins_r = go.Contour(x=x_vis_0_range,
                        y=x_vis_1_range,
                        z=YY_vis_r,
                        contours_coloring='lines',
                        line_width=2,
                        contours=dict(
                                      start=-1,
                                      end=1,
                                      size=2),
                        line=dict(dash='dash'),
                        colorscale=decision_colorscale,
                        showscale=False
                       )

fig5 = go.Figure(data=[margins_r, decision_surface_r, support_vectors_r, points], layout=layout)
fig5.show()

ValueError: X.shape[1] = 2 should be equal to 31, the number of features at training time

## Model selection

Let's pick the best model then. We will use the validation data for that. Let's predict using `svm` and `X_train` and assign it the name `yhat_train`. Also, predict `X_validation` and assign it the name `yhat_validation` (. The closeness of the accuracy of predictions on these two datasets will be helpful to us):

In [27]:
### begin your code here (2 lines).
yhat_train = svm.predict(X_train)
yhat_validation = svm.predict(X_validation)
### end your code here.

Let's measure the accuracy:

In [35]:
accuracy_score(yhat_train, y_train)

0.8979591836734694

In [36]:
accuracy_score(yhat_validation, y_validation)

0.75

We got ??.??% and ??.?%.

Let's repeat thge predictions for `svm_p2` and put the results in `yhat_train_p2` and `yhat_validation_p2`:

In [37]:
### begin your code here (2 lines).
yhat_train_p2 = svm_p2.predict(X_train)
yhat_validation_p2 = svm_p2.predict(X_validation)
### end your code here.

We can calculate the accuracies:

In [38]:
accuracy_score(yhat_train_p2, y_train)

0.5102040816326531

In [39]:
accuracy_score(yhat_validation_p2, y_validation)

0.375

??.??% and ??.??%.

Let's try predicting with `svm_p3` and put it in `yhat_train_p3` and `yhat_validation_p3`:

In [40]:
### begin your code here (2 lines).
yhat_train_p3 = svm_p3.predict(X_train)
yhat_validation_p3 = svm_p3.predict(X_validation)
### end your code here.

Now, if we predict the accuracy on these:

In [41]:
accuracy_score(yhat_train_p3, y_train)

0.5102040816326531

In [43]:
accuracy_score(yhat_validation_p3, y_validation)

0.5

The accuracy is ??.??% and ??.??%.

Finally, let's predict `yhat_train_r` and `yhat_validatin_r` using `svm_r`:

In [44]:
### begin your code here (2 lines).
yhat_train_r = svm_r.predict(X_train)
yhat_validation_r = svm_r.predict(X_validation)
### end your code here.

We can measure the accuracy of the SVM with RBF kernel:

In [45]:
accuracy_score(yhat_train_r, y_train)

0.5102040816326531

In [46]:
accuracy_score(yhat_validation_r, y_validation)

0.5

We got ??.??% and ??.??% when using the RBF kernel.

From all these number we can see that the RBF model works best as the accuracy on validation data is high and also the gap between the accuracy on training and validation data is not big. We can further tune the generalization power of our model by tuning the argument `C` of `SVC` which is the inverse of a regularization coefficient. We won't do that here now though. 

## Final assessment

Finally, let's check accuracy on the test data to get a final performance number. Predict `yhat_test_r` from `X_test` on `svm_r`:

In [48]:
### begin your code here (1 line).
yhat_test_r = svm_r.predict(X_test)
### end your code here.
accuracy_score(yhat_test_r, y_test)

0.5882352941176471

??.??%. We have good performance of test data.